In [211]:
#Importação das bibliotecas necessárias
import pandas as pd

from sklearn import tree
from sklearn.metrics import accuracy_score 
from sklearn.model_selection import train_test_split 

In [212]:
#Método que irá gerar métrica para classificação das casas
def house_class(n):
    if (n<-0.5):
        return 0
    elif (n<0.5):
        return 1
    else:
        return 2

In [213]:
#Leitura do csv com as informações das casas
data = pd.read_csv('kc_house_data.csv', sep=',')

print(data)

               id             date     price  bedrooms  bathrooms  \
0      7129300520  20141013T000000  221900.0         3       1.00   
1      6414100192  20141209T000000  538000.0         3       2.25   
2      5631500400  20150225T000000  180000.0         2       1.00   
3      2487200875  20141209T000000  604000.0         4       3.00   
4      1954400510  20150218T000000  510000.0         3       2.00   
...           ...              ...       ...       ...        ...   
21608   263000018  20140521T000000  360000.0         3       2.50   
21609  6600060120  20150223T000000  400000.0         4       2.50   
21610  1523300141  20140623T000000  402101.0         2       0.75   
21611   291310100  20150116T000000  400000.0         3       2.50   
21612  1523300157  20141015T000000  325000.0         2       0.75   

       sqft_living  sqft_lot  floors  waterfront  view  ...  grade  \
0             1180      5650     1.0           0     0  ...      7   
1             2570      7242   

In [214]:
#Filtragem dos dados pelo zipcode
filtered_data = data.query('zipcode==98008')
filtered_data = filtered_data.reset_index(drop=True)

print(filtered_data)

             id             date      price  bedrooms  bathrooms  sqft_living  \
0    7922800400  20140827T000000   951000.0         5       3.25         3250   
1    4036801170  20141013T000000   380000.0         4       1.75         1760   
2    8857320070  20140917T000000   490000.0         3       2.75         1980   
3    7375300160  20150309T000000   530000.0         5       2.25         2720   
4    4040800810  20140502T000000   420000.0         3       2.25         2000   
..          ...              ...        ...       ...        ...          ...   
278   224059021  20141219T000000   450000.0         3       1.00         1150   
279  8563010130  20140725T000000  1300000.0         3       2.50         3350   
280  8043700105  20150417T000000  2300000.0         4       4.00         4360   
281  7430500110  20141209T000000  1378000.0         5       3.50         5150   
282  9253900271  20150107T000000  3567000.0         5       4.50         4850   

     sqft_lot  floors  wate

In [215]:
#criando coluna com preço/metro²
filtered_data ['price_sqft'] = filtered_data['price']/filtered_data['sqft_living']

print(filtered_data)

             id             date      price  bedrooms  bathrooms  sqft_living  \
0    7922800400  20140827T000000   951000.0         5       3.25         3250   
1    4036801170  20141013T000000   380000.0         4       1.75         1760   
2    8857320070  20140917T000000   490000.0         3       2.75         1980   
3    7375300160  20150309T000000   530000.0         5       2.25         2720   
4    4040800810  20140502T000000   420000.0         3       2.25         2000   
..          ...              ...        ...       ...        ...          ...   
278   224059021  20141219T000000   450000.0         3       1.00         1150   
279  8563010130  20140725T000000  1300000.0         3       2.50         3350   
280  8043700105  20150417T000000  2300000.0         4       4.00         4360   
281  7430500110  20141209T000000  1378000.0         5       3.50         5150   
282  9253900271  20150107T000000  3567000.0         5       4.50         4850   

     sqft_lot  floors  wate

In [217]:
#cálculo para zscore
avg_prc = filtered_data['price_sqft'].mean()
print(avg_prc)
std_prc = filtered_data['price_sqft'].std()
print(std_prc)

zscore = (filtered_data['price_sqft'] - avg_prc) / std_prc

#adicionando coluna do zscore
filtered_data['zscore'] = zscore

print(filtered_data)

301.71773117246244
81.21573847793498
             id             date      price  bedrooms  bathrooms  sqft_living  \
0    7922800400  20140827T000000   951000.0         5       3.25         3250   
1    4036801170  20141013T000000   380000.0         4       1.75         1760   
2    8857320070  20140917T000000   490000.0         3       2.75         1980   
3    7375300160  20150309T000000   530000.0         5       2.25         2720   
4    4040800810  20140502T000000   420000.0         3       2.25         2000   
..          ...              ...        ...       ...        ...          ...   
278   224059021  20141219T000000   450000.0         3       1.00         1150   
279  8563010130  20140725T000000  1300000.0         3       2.50         3350   
280  8043700105  20150417T000000  2300000.0         4       4.00         4360   
281  7430500110  20141209T000000  1378000.0         5       3.50         5150   
282  9253900271  20150107T000000  3567000.0         5       4.50        

In [218]:
#aplicando o zscore para criação da coluna de avaliação
filtered_data['pricing'] = filtered_data['zscore'].apply(house_class)

print(filtered_data)

#salva arquivo csv filtrado e tratado
filtered_data.to_csv(r'filtered_data.csv', index=False)

#lista dos 10 ultimos valores
aval_data = filtered_data.tail(10)

print(aval_data)

#salva arquivo csv
aval_data.to_csv(r'aval_data.csv', index=False)

             id             date      price  bedrooms  bathrooms  sqft_living  \
0    7922800400  20140827T000000   951000.0         5       3.25         3250   
1    4036801170  20141013T000000   380000.0         4       1.75         1760   
2    8857320070  20140917T000000   490000.0         3       2.75         1980   
3    7375300160  20150309T000000   530000.0         5       2.25         2720   
4    4040800810  20140502T000000   420000.0         3       2.25         2000   
..          ...              ...        ...       ...        ...          ...   
278   224059021  20141219T000000   450000.0         3       1.00         1150   
279  8563010130  20140725T000000  1300000.0         3       2.50         3350   
280  8043700105  20150417T000000  2300000.0         4       4.00         4360   
281  7430500110  20141209T000000  1378000.0         5       3.50         5150   
282  9253900271  20150107T000000  3567000.0         5       4.50         4850   

     sqft_lot  floors  wate

In [207]:
#criação da árvore
dec_tree = tree.DecisionTreeClassifier(min_samples_leaf=10, min_impurity_decrease=0.001, min_samples_split=20)
x = filtered_data[['bedrooms', 'bathrooms', 'floors', 'view', 'condition']]
y = filtered_data['pricing']
x_train,x_test,y_train,y_test = train_test_split(x, y, test_size=0.2)
dec_tree.fit(x_train, y_train)


with open("decision_tree.dot", 'w') as f:
    f = tree.export_graphviz(dec_tree, out_file=f, feature_names=x.columns, rounded=True, filled=True, special_characters=True)
print('.dot gerado')

.dot gerado


In [208]:
#Teste de accuracy
predictions=dec_tree.predict(x_test)
print(accuracy_score(y_test,predictions))

0.543859649122807


In [209]:
#Comparação das 10 últimas casas da lista

predict = list(dec_tree.predict(x[-10:]))
print(predict)

ten_last = list(filtered_data['pricing'][-10:])
print(ten_last)

[0, 1, 1, 1, 1, 2, 0, 2, 1, 2]
[0, 2, 1, 1, 0, 2, 2, 2, 1, 2]
